In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, TimeDistributed, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Function to clean and preprocess data
def clean_and_save_data(csv_path, output_path):
    data = pd.read_csv(csv_path)
    # Check for valid pixel data
    data['pixel_length'] = data['pixels'].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)
    valid_data = data[data['pixel_length'] == 2304]  # Keep rows with 48x48 = 2304 pixels
    valid_data = valid_data.drop(columns=['pixel_length'])
    valid_data.to_csv(output_path, index=False)
    print("Cleaned dataset saved at:", output_path)

# Function to load and preprocess data from CSV
def load_data_from_csv(csv_path, sequence_length=10, img_height=48, img_width=48, num_classes=7):
    data = pd.read_csv(csv_path)
    X, y = [], []
    for i in range(0, len(data) - sequence_length, sequence_length):
        sequence = []
        valid_sequence = True
        for j in range(sequence_length):
            pixels = data.iloc[i + j]['pixels']
            if pd.isnull(pixels) or len(pixels.split()) != 2304:
                valid_sequence = False
                break
            pixels = np.array([int(p) for p in pixels.split()]).reshape(img_height, img_width, 1)
            sequence.append(pixels)
        if valid_sequence:
            X.append(sequence)
            y.append(data.iloc[i]['emotion'])
    X = np.array(X) / 255.0  # Normalize pixel values to [0, 1]
    y = tf.keras.utils.to_categorical(np.array(y), num_classes=num_classes)  # One-hot encode labels
    return X, y

# Paths
input_csv_path = 'Dataset/fer2013.csv'
cleaned_csv_path = 'Dataset/fer2013_cleaned.csv'

# Step 1: Clean and save the data
clean_and_save_data(input_csv_path, cleaned_csv_path)

# Step 2: Load cleaned data
X, y = load_data_from_csv(cleaned_csv_path)

# Step 3: Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Step 4: Define the CNN-LSTM model
num_classes = 7  # FER2013 has 7 classes of emotions
model = Sequential()

# TimeDistributed CNN for feature extraction from each frame
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(None, 48, 48, 1)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Dropout(0.25)))

model.add(TimeDistributed(Flatten()))

# LSTM for temporal feature learning
model.add(LSTM(128, activation='tanh', return_sequences=False))
model.add(Dropout(0.5))

# Fully connected layer for classification
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Multi-class classification

# Compile the model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# Step 5: Train the model
if len(X_train) > 0 and len(y_train) > 0:
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, lr_scheduler])

    # Step 6: Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.show()

    # Step 7: Evaluate the model on the test set
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    y_true = np.argmax(y_test, axis=-1)
    print("Test Accuracy:", accuracy_score(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, cmap='Blues', interpolation='nearest')
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print("Not enough data to train the model.")

# Step 8: Visualize sample images (Optional)
for i in range(5):
    img = X[i][0].reshape(48, 48)  # First frame of the sequence
    plt.imshow(img, cmap='gray')
    plt.title(f"Emotion: {np.argmax(y[i])}")
    plt.show()
